In [2]:
%reload_ext autoreload
%autoreload 2

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

import os, sys
from utils.sunrgbd_config import SUNRGBD_CONFIG
from configs.data_config import NYU40CLASSES
from utils.sunrgbd_utils import readsunrgbdframe, process_sunrgbd_frame, check_bdb
from utils.vis_tools_sunrgbd import Scene3D_SUNRGBD
import os
import numpy as np
import pickle
from configs.data_config import Config
from utils.sunrgbd_utils import proj_from_point_to_2d, get_corners_of_bb3d_no_index
from net_utils.libs import get_iou
from libs.tools import camera_cls_reg_sunrgbd, layout_size_avg_residual, ori_cls_reg, obj_size_avg_residual, bin_cls_reg, list_of_dict_to_dict_of_list
import json

from utils.utils_rui import *

sunrgbd_config = SUNRGBD_CONFIG()
image_id = 3
sequence = readsunrgbdframe(sunrgbd_config, image_id=image_id) # utils.sunrgbd_utils -> class SUNRGBDData()
scene = Scene3D_SUNRGBD(sequence) # utils.vis_tools_sunrgbd -> class Scene3D_SUNRGBD(Scene3D)

# scene.draw_image()
# scene.draw_cls()
# scene.draw_inst()
im, color_list = scene.draw_projected_bdb3d(return_plt=True)

%matplotlib widget

import matplotlib.pyplot as plt
fig = plt.figure(figsize=(15, 5))
plt.subplot(121)
plt.imshow(im)

im, bdb2D_from_3D = scene.draw_projected_layoutbdb3d(return_plt=True, fix_polygon=True)
plt.subplot(122)
plt.imshow(im)
# scene.draw3D()

# display(fig)
plt.show()

print(sequence.R_ex)

[[ 0.99992967  0.          0.01185941]
 [ 0.          1.          0.        ]
 [-0.01185941  0.          0.99992967]]
(530, 730, 3)
[draw_projected_bdb3d] Returned.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[draw_projected_layoutbdb3d] Returned.
[[ 0.990197  0.139566  0.005576]
 [-0.139566  0.987025  0.079386]
 [ 0.005576 -0.079386  0.996828]]


In [77]:
scene.cam_K

array([[529.5,   0. , 365. ],
       [  0. , 529.5, 265. ],
       [  0. ,   0. ,   1. ]])

## what happens in 'class Scene3D_SUNRGBD(Scene3D)'

In [67]:
np.set_printoptions(precision=4)
np.set_printoptions(suppress=True)

from utils.sunrgbd_utils import get_NYU37_class_id, cvt2nyu37class_map, process_layout, get_world_R, transform_to_world,\
    cvt_R_ex_to_cam_R, check_bdb2d, process_bdb3d, process_bdb2d, get_inst_map, normalize_point, get_campact_layout

cam_R = cvt_R_ex_to_cam_R(sequence.R_ex) # transform objects from world system to camera system
# define a world system
world_R = get_world_R(cam_R) # almost identity matrix
print(cam_R)
print(world_R)

layout, layout_t = process_layout(sequence.manhattan_layout) # sequence.manhattan_layout: 8-point bbox; transform sunrgbd layout to toward-up-right form.
centroid = layout['centroid']
vectors = np.diag(layout['coeffs']).dot(layout['basis'])

bdb3ds_ws = process_bdb3d(sequence.bdb3d) # sequence.bdb3d: an array of length: num of objects

# key: transform everything to world system
layout, __bdb3d, __cam_R = transform_to_world(layout, bdb3ds_ws, cam_R, world_R)

fig = plt.figure()
ax = fig.gca(projection='3d')
ax.set_aspect("auto")
vis_axis(ax)

# bdb3d_layout_not_processed = get_corners_of_bb3d_no_index(layout['basis'], layout['coeffs'], layout['centroid'])
bdb3d_layout_not_processed = layout_t
vis_cube_plt(bdb3d_layout_not_processed, ax, 'b')

# comparing against: processed inside the function
bdb3d_layout = get_corners_of_bb3d_no_index(scene.layout['basis'], scene.layout['coeffs'], scene.layout['centroid'])
vis_cube_plt(bdb3d_layout, ax, 'r')

ax.view_init(elev=-66, azim=92)

[[ 0.8746  0.4838  0.0304]
 [-0.4838  0.8673  0.1174]
 [ 0.0304 -0.1174  0.9926]]
[[ 0.9994  0.     -0.0348]
 [ 0.      1.      0.    ]
 [ 0.0348  0.      0.9994]]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [65]:
sequence.manhattan_layout

array([[ 0.4837, -1.3289, -1.6843],
       [ 2.649 ,  3.3146, -1.6843],
       [-1.174 ,  5.0973, -1.6843],
       [-3.3393,  0.4538, -1.6843],
       [ 0.4837, -1.3289,  4.    ],
       [ 2.649 ,  3.3146,  4.    ],
       [-1.174 ,  5.0973,  4.    ],
       [-3.3393,  0.4538,  4.    ]], dtype=float32)

In [61]:
__cam_R

array([[ 0.8752,  0.4794,  0.0649],
       [-0.4838,  0.8673,  0.1174],
       [ 0.    , -0.1342,  0.991 ]])

In [40]:
bdb3ds_ws[0], __bdb3d[0]

({'coeffs': array([0.8753, 0.4823, 1.0979]),
  'basis': array([[-0.6221,  0.    ,  0.7829],
         [ 0.    ,  1.    ,  0.    ],
         [-0.7829, -0.    , -0.6221]]),
  'centroid': array([ 2.5768, -0.7177,  0.3023]),
  'class_id': 4},
 {'coeffs': array([0.8753, 0.4823, 1.0979]),
  'basis': array([[-0.6178,  0.    ,  0.7863],
         [ 0.    ,  1.    ,  0.    ],
         [-0.7863,  0.    , -0.6178]]),
  'centroid': array([ 2.5784, -0.7177,  0.2881]),
  'class_id': 4})

In [29]:
cam_R, world_R

(array([[ 0.934 ,  0.3541,  0.0482],
        [-0.3541,  0.8988,  0.2585],
        [ 0.0482, -0.2585,  0.9648]]),
 array([[ 0.9987,  0.    , -0.0515],
        [ 0.    ,  1.    ,  0.    ],
        [ 0.0515,  0.    ,  0.9987]]))

In [46]:
bdb3d_layout_not_processed

array([[-1.2711,  4.    , -0.62  ],
       [ 0.3442,  4.    ,  4.071 ],
       [ 5.2405,  4.    ,  2.3851],
       [ 3.6253,  4.    , -2.3059],
       [-1.2711, -1.2711, -0.62  ],
       [ 0.3442, -1.2711,  4.071 ],
       [ 5.2405, -1.2711,  2.3851],
       [ 3.6253, -1.2711, -2.3059]])

In [47]:
bdb3d_layout

array([[-1.2764,  4.    , -0.6089],
       [ 0.3796,  4.    ,  4.0679],
       [ 5.2611,  4.    ,  2.3394],
       [ 3.605 ,  4.    , -2.3374],
       [-1.2764, -1.2711, -0.6089],
       [ 0.3796, -1.2711,  4.0679],
       [ 5.2611, -1.2711,  2.3394],
       [ 3.605 , -1.2711, -2.3374]])

In [15]:
print(scene.layout['basis'], layout['basis'])

[[ 0.926   0.      0.3775]
 [ 0.      1.      0.    ]
 [-0.3775  0.      0.926 ]] [[ 0.9205  0.      0.3907]
 [ 0.      1.      0.    ]
 [-0.3907  0.      0.9205]]


In [16]:
print(scene.layout['coeffs'], layout['coeffs'])

[2.2679 2.6875 1.7295] [2.2679 2.6875 1.7295]


In [17]:
print(scene.layout['centroid'], layout['centroid'])

[ 1.4495  1.3125 -0.1969] [ 1.4522  1.3125 -0.1761]


# vis from 'scene'

In [34]:
fig = plt.figure()

ax = fig.gca(projection='3d')
ax.set_aspect("auto")
vis_axis(ax)

# vis_cube_plt(data_frame.manhattan_layout, ax)
bdb3d_layout = get_corners_of_bb3d_no_index(scene.layout['basis'], scene.layout['coeffs'], scene.layout['centroid'])
vis_cube_plt(bdb3d_layout, ax, 'r')
ax.view_init(elev=-66, azim=92)

from utils.sunrgbd_utils import proj_from_point_to_2d

for bdb3d, color in zip(scene.bdb3d, color_list):
    bdb3d_corners = get_corners_of_bb3d_no_index(bdb3d['basis'], bdb3d['coeffs'], bdb3d['centroid']) # bbox y axis is absolute upright
    vis_cube_plt(bdb3d_corners, ax, color=color)
    
for bdb3d, color in zip(bdb3ds_ws, color_list):
    bdb3d_corners = get_corners_of_bb3d_no_index(bdb3d['basis'], bdb3d['coeffs'], bdb3d['centroid']) # bbox y axis is absolute upright
    vis_cube_plt(bdb3d_corners, ax)

fig.canvas.toolbar_visible = True

# display(fig)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[0.4, 0.3686274509803922, 0.9137254901960784]
[0.23529411764705882, 0.7647058823529411, 0.6627450980392157]
[0.7294117647058823, 0.7450980392156863, 0.47843137254901963]
[0.611764705882353, 0.03529411764705882, 0.396078431372549]


In [9]:
sequence.bdb3d[0]

(array([[ 0.7829, -0.6221,  0.    ],
       [ 0.6221,  0.7829,  0.    ],
       [ 0.    ,  0.    ,  1.    ]]), array([[0.8753, 1.0979, 0.4823]]), array([[ 0.3023,  2.5768, -0.7177]]), array(['bed'], dtype='<U3'), array(['SUNRGBD/kv2/kinect2data/000015_2014-05-26_14-37-33_260595134347_rgbf000020-resize'],
      dtype='<U81'), array([[-0.6221, -0.7829,  0.    ]]), array([[1]], dtype=uint8))

In [35]:
bdb3ds_ws

[{'coeffs': array([0.8753, 0.4823, 1.0979]),
  'basis': array([[-0.6221,  0.    ,  0.7829],
         [ 0.    ,  1.    ,  0.    ],
         [-0.7829, -0.    , -0.6221]]),
  'centroid': array([ 2.5768, -0.7177,  0.3023]),
  'class_id': 4},
 {'coeffs': array([0.3175, 0.3554, 0.2273]),
  'basis': array([[-0.6317,  0.    ,  0.7752],
         [ 0.    ,  1.    ,  0.    ],
         [-0.7752, -0.    , -0.6317]]),
  'centroid': array([ 4.0484, -0.8309, -0.1426]),
  'class_id': 32},
 {'coeffs': array([0.2995, 0.3256, 0.2808]),
  'basis': array([[-0.534 ,  0.    ,  0.8455],
         [ 0.    ,  1.    ,  0.    ],
         [-0.8455, -0.    , -0.534 ]]),
  'centroid': array([ 2.4155, -0.8744,  1.8649]),
  'class_id': 32},
 {'coeffs': array([0.193 , 0.3727, 0.193 ]),
  'basis': array([[-0.6832,  0.    ,  0.7303],
         [ 0.    ,  1.    ,  0.    ],
         [-0.7303, -0.    , -0.6832]]),
  'centroid': array([ 4.    , -0.0955, -0.0955]),
  'class_id': 35}]

In [62]:
scene.layout

{'centroid': array([ 1.4042413 ,  1.34581369, -0.38624769]),
 'coeffs': array([2.30113326, 2.65418631, 2.21155161]),
 'basis': array([[ 0.98055391,  0.        ,  0.19624992],
        [ 0.        ,  1.        ,  0.        ],
        [-0.19624992,  0.        ,  0.98055391]])}

In [55]:
center_from_3D

array([[-1.26924317e+04, -1.33977421e+07]])

In [46]:
data_frame.bdb3d[1]

(array([[ 0.96835519, -0.24957608,  0.        ],
       [ 0.24783325,  0.9688027 ,  0.        ],
       [ 0.        ,  0.        ,  1.        ]]), array([[0.45907792, 0.34032744, 0.393974  ]]), array([[ 1.30276515,  3.05095168, -0.86269307]]), array(['night_stand'], dtype='<U11'), array(['SUNRGBD/kv2/kinect2data/000003_2014-05-26_14-24-42_260595134347_rgbf000040-resize'],
      dtype='<U81'), array([[-0.24783325, -0.9688027 ,  0.        ]]), array([[1]], dtype=uint8))

In [30]:
# # draw sphere
# u, v = np.mgrid[0:2*np.pi:20j, 0:np.pi:10j]
# x = np.cos(u)*np.sin(v)
# y = np.sin(u)*np.sin(v)
# z = np.cos(v)
# ax.plot_wireframe(x, y, z, color="r")

# # draw a point
# ax.scatter([0], [0], [0], color="g", s=100)

# # draw a vector
# from matplotlib.patches import FancyArrowPatch
# from mpl_toolkits.mplot3d import proj3d


# class Arrow3D(FancyArrowPatch):

#     def __init__(self, xs, ys, zs, *args, **kwargs):
#         FancyArrowPatch.__init__(self, (0, 0), (0, 0), *args, **kwargs)
#         self._verts3d = xs, ys, zs

#     def draw(self, renderer):
#         xs3d, ys3d, zs3d = self._verts3d
#         xs, ys, zs = proj3d.proj_transform(xs3d, ys3d, zs3d, renderer.M)
#         self.set_positions((xs[0], ys[0]), (xs[1], ys[1]))
#         FancyArrowPatch.draw(self, renderer)

# a = Arrow3D([0, 1], [0, 1], [0, 1], mutation_scale=20,
#             lw=1, arrowstyle="-|>", color="k")
# ax.add_artist(a)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …